In [1]:
import os, shutil

In [2]:
original_tortilla_dir = '/home/dsc/tfm_data/tortilla_de_patata'
original_croquetas_dir = '/home/dsc/tfm_data/croquetas'

In [3]:
!pwd

/home/dsc/TFM_food_recognition


In [4]:
base_dir = '/home/dsc/tfm_data/tortilla_and_croquetas'
os.mkdir(base_dir)

Creamos los directorios de training, validation y test:

In [5]:
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

Creamos las carpetas de train, validation y test de cada plato:

In [6]:
train_tortilla_dir = os.path.join(train_dir, 'tortilla')
os.mkdir(train_tortilla_dir)
train_croquetas_dir = os.path.join(train_dir, 'croquetas')
os.mkdir(train_croquetas_dir)

In [8]:
validation_tortilla_dir = os.path.join(validation_dir, 'tortilla')
os.mkdir(validation_tortilla_dir)
validation_croquetas_dir = os.path.join(validation_dir, 'croquetas')
os.mkdir(validation_croquetas_dir)

In [9]:
test_tortilla_dir = os.path.join(test_dir, 'tortilla')
os.mkdir(test_tortilla_dir)
test_croquetas_dir = os.path.join(test_dir, 'croquetas')
os.mkdir(test_croquetas_dir)

In [ ]:
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

Vamos a rellenar los directorios creados anteriormente con las fotos de los platos. Para el caso de las croquetas, vamos a seleccionar un total de 1100 fotos, de las cuales 550 fotos serán para el train, 275 para el validation y 275 para el test.

In [20]:
# Fotos de train:
fotos = os.listdir(original_croquetas_dir)[0:550]
for foto in fotos:
    src = os.path.join(original_croquetas_dir, foto)
    dst = os.path.join(train_croquetas_dir, foto)
    shutil.copyfile(src, dst)

In [21]:
# Fotos de validation:
fotos = os.listdir(original_croquetas_dir)[550:825]
for foto in fotos:
    src = os.path.join(original_croquetas_dir, foto)
    dst = os.path.join(validation_croquetas_dir, foto)
    shutil.copyfile(src, dst)

In [22]:
# Fotos de test:
fotos = os.listdir(original_croquetas_dir)[825:1101]
for foto in fotos:
    src = os.path.join(original_croquetas_dir, foto)
    dst = os.path.join(test_croquetas_dir, foto)
    shutil.copyfile(src, dst)

Para las tortillas de patata hemos seleccionado 1200 imágenes. Para el set de train se usarán 600 imágenes y para los set de validation y test serán 300 fotos respectivamente.

In [23]:
# Fotos de train:
fotos = os.listdir(original_tortilla_dir)[0:600]
for foto in fotos:
    src = os.path.join(original_tortilla_dir, foto)
    dst = os.path.join(train_tortilla_dir, foto)
    shutil.copyfile(src, dst)

In [24]:
# Fotos de validation:
fotos = os.listdir(original_tortilla_dir)[600:900]
for foto in fotos:
    src = os.path.join(original_tortilla_dir, foto)
    dst = os.path.join(validation_tortilla_dir, foto)
    shutil.copyfile(src, dst)

In [25]:
# Fotos de test:
fotos = os.listdir(original_tortilla_dir)[900:1200]
for foto in fotos:
    src = os.path.join(original_tortilla_dir, foto)
    dst = os.path.join(test_tortilla_dir, foto)
    shutil.copyfile(src, dst)

Una vez preparadas las fotos en sus respectivos directorios, vamos a realizar una pequeña convnet:

In [26]:
from keras import layers
from keras import models

In [27]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

Configuramos el modelo para el training;

In [29]:
from keras import optimizers
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

Using <code>ImageDataGenerator</code> para leer las imágeenes de los directorios:

In [31]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255) # reescalar las imagenes por 1/255
test_datagen = ImageDataGenerator(rescale=1./255)

In [32]:
train_generator =train_datagen.flow_from_directory(
        train_dir, # directorio objetivo
        target_size=(150,150), # rescalamos las imagenes a 150 x 150
        batch_size=20,
        class_mode='binary' # hemos usado binary_crossentropy, por lo que necesitamos etiquetas binary
)

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150,150), 
        batch_size=20,
        class_mode='binary'
)


Found 1150 images belonging to 2 classes.
Found 575 images belonging to 2 classes.


In [33]:
for data_batch, labels_batch in train_generator:
    print('data_batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data_batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


Ajustando el modelo con un batch generator:

In [34]:
history = model.fit_generator(
    train_generator, 
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator, 
    validation_steps=50)

/home/dsc/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
100/100 [==============================] - 31s 302ms/step - loss: 0.6756 - acc: 0.5895 - val_loss: 0.6979 - val_acc: 0.5322


In [35]:
history = model.fit(
    train_generator, 
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator, 
    validation_steps=50)

Epoch 1/30
100/100 [==============================] - 31s 309ms/step - loss: 0.5483 - acc: 0.7322 - val_loss: 0.5141 - val_acc: 0.7565
